# Análisis de Predicción de Churn para Interconnect (Empresa de Telecomunicaciones)

Este notebook contiene el análisis completo de predicción de churn (abandono de clientes) siguiendo la metodología CRISP-DM (Cross-Industry Standard Process for Data Mining).

## a) Business Understanding

### Formulación del Problema

En la empresa **Interconnect**, el equipo de marketing enfrenta un desafío significativo con la pérdida de clientes (churn), lo cual impacta directamente en los ingresos y la rentabilidad. La adquisición de nuevos clientes suele costar de 5 a 25 veces más que retener a los existentes, por lo que identificar clientes con alto riesgo de abandono es crucial para implementar estrategias de retención efectivas.

El equipo de marketing desea anticiparse a las cancelaciones de suscripciones para ofrecer códigos promocionales específicamente a clientes que muestren señales de posible abandono, optimizando así el presupuesto de retención.

Para este proyecto, contamos con cuatro fuentes de datos interconectadas mediante el identificador único `customerID`:

1. **contract.csv** - Detalles del contrato (tipo, duración, método de pago, importe, etc.)
2. **personal.csv** - Datos demográficos y segmentación del cliente
3. **internet.csv** - Servicios de Internet y características técnicas
4. **phone.csv** - Servicios telefónicos y líneas asociadas

**Objetivo principal**: Desarrollar un modelo de machine learning que prediga con precisión qué clientes tienen mayor probabilidad de abandonar el servicio, permitiendo al equipo de marketing dirigir eficientemente los códigos promocionales a aquellos clientes en riesgo de abandono.

### Hipótesis

Basándonos en el conocimiento del dominio de telecomunicaciones y las fuentes de datos disponibles, planteamos las siguientes hipótesis que guiarán nuestra investigación:

1. **H1**: Los clientes con contratos mensuales tienen mayor probabilidad de abandono que aquellos con contratos a largo plazo (verificable mediante `contract.csv`).

2. **H2**: El tiempo de permanencia como cliente está inversamente relacionado con la probabilidad de abandono (clientes más nuevos tienen mayor riesgo), información disponible en `contract.csv`.

3. **H3**: Los clientes con múltiples líneas telefónicas o servicios adicionales (`phone.csv` y `internet.csv`) tienen menor probabilidad de abandono debido al mayor costo de cambio.

4. **H4**: Los clientes con gastos mensuales más altos son más sensibles a la calidad del servicio y tienen mayor probabilidad de abandono si no perciben un valor adecuado (verificable combinando información financiera de `contract.csv`).

5. **H5**: El uso de múltiples servicios de Internet, como streaming y seguridad (`internet.csv`), reduce la probabilidad de abandono por el efecto de "cliente integral".

6. **H6**: Los métodos de pago automáticos (débito bancario) están asociados a menor tasa de abandono debido a la menor fricción en los pagos (información presente en `contract.csv`).

7. **H7**: Existen patrones demográficos significativos (`personal.csv`) que pueden indicar mayor propensión al abandono en ciertos segmentos de clientes.

Estas hipótesis serán evaluadas durante las fases de análisis exploratorio y modelado, utilizando las cuatro fuentes de datos disponibles.

### Key Performance Indicators (KPIs)

Para medir el éxito de nuestro modelo predictivo y las posteriores estrategias de retención basadas en códigos promocionales, definimos los siguientes KPIs:

#### KPIs del modelo:

1. **AUC-ROC**: El área bajo la curva ROC deberá ser superior a 0.85 para considerar el modelo adecuado para implementación.

2. **Recall (Sensibilidad)**: Deberá ser superior al 80%, priorizando la identificación correcta de clientes que abandonarán (minimizando falsos negativos), para garantizar que no se pierdan oportunidades de retención.

3. **Precisión a diferentes umbrales**: Evaluaremos la precisión del modelo en diferentes puntos de corte de probabilidad para optimizar el balance entre recall y precisión según el presupuesto disponible para códigos promocionales.

#### KPIs de negocio (post-implementación):

1. **Efectividad de los códigos promocionales**: Tasa de redención de códigos promocionales de al menos un 30% entre los clientes identificados como de alto riesgo.

2. **Reducción de la tasa de churn**: Disminución de al menos un 20% en la tasa de abandono entre los clientes identificados como de alto riesgo y contactados con promociones.

3. **ROI de las campañas de retención**: Por cada dólar invertido en códigos promocionales dirigidos por el modelo, se espera un retorno de al menos 3 dólares en ingresos preservados.

4. **Eficiencia presupuestaria**: Reducción del 30% en el gasto promocional por cliente retenido al focalizar esfuerzos en segmentos con mayor riesgo de abandono, comparado con campañas masivas anteriores.

Estos KPIs serán monitoreados continuamente para evaluar la efectividad tanto del modelo predictivo como de la estrategia de retención basada en códigos promocionales.

In [1]:
# Importación de librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import shap
import warnings
from pathlib import Path
import joblib

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

# Ignorar advertencias para una salida más limpia
warnings.filterwarnings('ignore')

# Configurar semilla para reproducibilidad
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuración de rutas para estructura del proyecto

# Definir rutas para los directorios del proyecto
base_dir = Path.cwd().parent

# Verificar y crear directorios si no existen
data_dir = base_dir / 'data'
models_dir = base_dir / 'models'
notebooks_dir = base_dir / 'notebooks'

for directory in [data_dir, models_dir, notebooks_dir]:
    if not directory.exists():
        directory.mkdir(parents=True)
        print(f"Directorio creado: {directory}")
    else:
        print(f"Directorio existente: {directory}")

print(f"\nEstructura de directorios:\n")
print(f"- Datos: {data_dir}")
print(f"- Modelos: {models_dir}")
print(f"- Notebooks: {notebooks_dir}")

Directorio existente: c:\Users\Rony\Desktop\OneDrive - Universidad San Francisco de Quito\USFQ\NovenoSemestre\DataMining\Examen Final\data
Directorio existente: c:\Users\Rony\Desktop\OneDrive - Universidad San Francisco de Quito\USFQ\NovenoSemestre\DataMining\Examen Final\models
Directorio existente: c:\Users\Rony\Desktop\OneDrive - Universidad San Francisco de Quito\USFQ\NovenoSemestre\DataMining\Examen Final\notebooks

Estructura de directorios:

- Datos: c:\Users\Rony\Desktop\OneDrive - Universidad San Francisco de Quito\USFQ\NovenoSemestre\DataMining\Examen Final\data
- Modelos: c:\Users\Rony\Desktop\OneDrive - Universidad San Francisco de Quito\USFQ\NovenoSemestre\DataMining\Examen Final\models
- Notebooks: c:\Users\Rony\Desktop\OneDrive - Universidad San Francisco de Quito\USFQ\NovenoSemestre\DataMining\Examen Final\notebooks
